# Example: Routing avoiding country borders and tollroads

This examples gives you an idea how to use the option and extra info parameter within the direction module of Openrouteservice. 

In [1]:
import folium
from folium import Map, Marker

from openrouteservice import client
from openrouteservice.directions import directions

We are working for a Transportation company and are receiving a job to carry goods from Munich to Budapest. Because we have drive this route in the future quite often it would be great to know the route differences if we are using the car or the HGV (heavy good vehicle). 
We want to avoid the Austrian border.


Bevor the actual routing begins we set up all needed dictionaries and the direction request.

## Routing using tollroads to save time

In [2]:
api_key = '58d904a497c67e00015b45fc2a6b6872037d44119582ef40cdf264c4' # Provide your personal API key
clnt = client.Client(key=api_key) # Create client with api key
map1 = folium.Map(tiles='Stamen Toner', location=([48.57479, 15.46875]), zoom_start=7) # Set up folium map

# Set up driving profile dictionary for direction request
profile_dict = {'hgv': {'profile': 'driving-hgv'},
                'car': {'profile': 'driving-car'}}

# Direction Request from Munich to Budapest with real coordinates
params_route = {'coordinates': [[11.544056, 48.151543], [19.092865, 47.514882]],
               'format_out': 'geojson',
               'geometry': 'true',
               'geometry_format': 'geojson',
               'preference': 'fastest',
                'instructions': 'true',
               'options': {'avoid_countries': '11'}} # Avoiding Austria

# Request of extra infos of tollroad parts
params_extrainfo = {'coordinates': [[11.544056, 48.151543], [19.092865, 47.514882]],
                      'geometry': 'true',
                      'preference': 'fastest',
                      'instructions': 'false',
                      'extra_info': {'tollways'},
                      'options': {'avoid_countries': '11'}} 

# Set up style dictionary for each driving route
style_dict = {'hgv': {'tollroad': '#0000cc', #blue
                      'tollfree': '#0099ff'}, #lightblue
              'car': {'tollroad': '#00802b', #green
                      'tollfree': '#73e600'}} #lightgreen

In [3]:
tollroad_array = [] # Tollroute data
toll_values = [] # Tollroad parts of tollroutes

for vehicle, profile in profile_dict.items():
    for style_vehicle, style in style_dict.items():
        if style_vehicle in vehicle:
            profile['style_tollroad'] = style['tollroad'] # Defining color of tollroads
    params_route['profile'] = profile['profile']
    tollroad_route = clnt.directions(**params_route)
    for avoid_country in tollroad_route['features']:
        coords_countries = [[y,x] for x,y in avoid_country['geometry']['coordinates']] # reverse coordinates due to weird folium lat/lon syntax
        folium.features.PolyLine(coords_countries, color=profile['style_tollroad'], popup='Tollroad').add_to(map1) # Tollroutes without marked tollparts
    tollroad_array.append(tollroad_route)

# Request tollroad parts
    params_extrainfo['profile'] = profile['profile']
    toll_route = clnt.directions(**params_extrainfo)
    for value in toll_route['routes']:
        toll_values += [value['extras']['tollways']['values']]

# Tollroad parts are just given as waypoints, why they have to be combined with coordinates
route_kombi_dict = {'hgv': [toll_values[0], tollroad_array[0]], 'car': [toll_values[1],  tollroad_array[1]]}

for vehicle in route_kombi_dict:
    for value in route_kombi_dict[vehicle][0]:
        if 1 in value: # 1 = tollroad, 0 = no tollroad
            vehicle_value = [value[0:2]]
            
            for i,j in vehicle_value:
                vehicle_coords = [coords['geometry']['coordinates'][i:j] for coords in route_kombi_dict[vehicle][1]['features']]
                               
                for coords in vehicle_coords:
                    switched_vehicle_coords = [[y,x] for x,y in coords] 
                    folium.PolyLine(switched_vehicle_coords, color='#cc0000', popup='Tollroad').add_to(map1) # Tollroad parts
map1

Okay, if choosing this route we would drive a lot of tollroad. Especially when using the car (green colored route). It could be quite expensive, so maybe we should think about avoiding tollroads.

## Routing avoiding tollroads to save money

To save some money, let's have a look how the routing would look like if we avoid the austrian border and tollroads.

In [4]:
tollfree_array = []
for vehicle, profile in profile_dict.items():
    for style_vehicle, style in style_dict.items():
        if style_vehicle in vehicle:
            profile['style_tollfree'] = style['tollfree'] # Defining color of tollfree roads
    params_route['profile'] = profile['profile']
    params_route['options'] = {'avoid_countries': '11', # Avoiding Austria and tollroads
                               'avoid_features': 'tollways'}
    tollfree_route = clnt.directions(**params_route)
    for avoid_toll in tollfree_route['features']:
        coords_tollfree = [[y,x] for x,y in avoid_toll['geometry']['coordinates']]
        folium.features.PolyLine(coords_tollfree, color=profile['style_tollfree'], popup='Avoiding tollroad').add_to(map1)
    tollfree_array.append(tollfree_route)

folium.map.Marker([48.151543, 11.544056], popup='Start: Munich').add_to(map1)
folium.map.Marker([47.514882, 19.092865], popup='Destination: Budapest').add_to(map1)

map1

That doesn't seem too bad, but we should have a look at real facts to know the correct distance and duration for each route. 

In [5]:
for tollroad_infos in tollroad_array:
    print('Tollroad information for {}: Distance: {:.2f} km and time: {:.2f} hours'.format(tollroad_infos['info']['query']['profile'], 
                                                                           tollroad_infos['features'][0]['properties']['summary'][0]['distance']/1000,
                                                                           tollroad_infos['features'][0]['properties']['summary'][0]['duration']/3600))

for tollfree_infos in tollfree_array:
    print('Tollfree road information for {}: Distance: {:.2f} km and time: {:.2f} hours'.format(tollfree_infos['info']['query']['profile'],
                                                                                               tollfree_infos['features'][0]['properties']['summary'][0]['distance']/1000,
                                                                                               tollfree_infos['features'][0]['properties']['summary'][0]['duration']/3600))

Tollroad information for driving-hgv: Distance: 819.89 km and time: 12.08 hours
Tollroad information for driving-car: Distance: 911.22 km and time: 8.44 hours
Tollfree road information for driving-hgv: Distance: 767.46 km and time: 13.04 hours
Tollfree road information for driving-car: Distance: 791.05 km and time: 11.20 hours


That makes the decision a lot easier. Using the HGV, it could be an option to use the tollfree route, because is just takes one hour longer, but using the car the tollroad is a good suggestion to save time, although the route distance is longer than using the tollfree route.